In [1]:
# Импорт библиотек
import os
import time

import GTU
import KU_TU
import mat_properties as prop
import numpy as n
import pandas as pd
import SP
from scipy.optimize import root

# таблица номинального режима
gas_streams0 = pd.read_excel("streams0.xlsx", sheet_name="gas", index_col=0)
water_streams0 = pd.read_excel("streams0.xlsx", sheet_name="water", index_col=0)
GTU_ISO = pd.read_excel("input.xlsx", sheet_name="ISO", index_col=0)
GTU_input = pd.read_excel("input.xlsx", sheet_name="GTU_input", index_col=0)
# рабочая таблица (=номинал в 1 итерации)
gas_streams = pd.read_excel("streams.xlsx", sheet_name="gas", index_col=0)
water_streams = pd.read_excel("streams.xlsx", sheet_name="water", index_col=0)
# рабочая таблица показателей блоков
heaters = pd.read_excel("blocks.xlsx", sheet_name="heaters", index_col=0)
electric = pd.read_excel("blocks.xlsx", sheet_name="electric", index_col=0)
############################################################
# Теплосеть и перекидка температуры воздуха
gas_streams.loc["AIR", "T":"P"] = [GTU_input.loc["tair", 1], 0.1]
water_streams.loc["AIR", "T":"P"] = [GTU_input.loc["tair", 1], 0.1]
Tnv = gas_streams.at["AIR", "T"]
water_streams.at["SWINB", "T"] = SP.Tset(Tnv)[1]
water_streams.at["SWOUT", "T"] = SP.Tset(Tnv)[0]
water_streams.at["SWIN-TURB", "T"] = water_streams.at["SWINB", "T"]
water_streams.at["SP2-WOUT", "T"] = water_streams.at["SWOUT", "T"]
############################################################


# Основные эффективности оборудования
KPD_PN = 0.8074
KPD_KN = 0.75
KPD_to = 0.99
KPD_SP = 0.99


# Параметры, отвечающие за процесс расчета
Calcmethod = "hybr"
Calctolerance = 10**-2
Maxiterations_KU_TU = 20
Maxiterations_cotel = 4
Maxiterations_turbine = 30

# Параметры режима работы ПГУ
# Расчет для минимума нагрузки
Сalculate_minimum = True
# Расчет для работы с теплофикацией
Teplo = int(False)
# Отбор пара высокго давления или доля или кг/с
steamVD_fraction_to_turbine = 1


# Состав газов в номинале в ГТУ
gasmix = "Nitrogen*Oxygen*CO2*Water*Argon"
# Считывание рефпропа
RP = prop.init_REFPROP(r"C:\Program Files (x86)\REFPROP")
fractiongas0 = list(gas_streams0.loc["GTU-PEVD", "N2":"Ar"])
gas0 = prop.Materials_prop(
    gasmix,
    fractiongas0,
    prop.REFPROP_h_s,
    prop.REFPROP_p_t,
    prop.REFPROP_p_h,
    prop.REFPROP_p_s,
    prop.REFPROP_p_q,
    prop.REFPROP_t_q,
    prop.REFPROP_p_rho,
    prop.REFPROP_s_q,
    RP=RP,
)
water = prop.Materials_prop(
    "water",
    [1.0, 0, 0, 0, 0],
    prop.REFPROP_h_s,
    prop.REFPROP_p_t,
    prop.REFPROP_p_h,
    prop.REFPROP_p_s,
    prop.REFPROP_p_q,
    prop.REFPROP_t_q,
    prop.REFPROP_p_rho,
    prop.REFPROP_s_q,
    RP=RP,
)
# Задание энтальпий газа в номинальном режиме
Temperatures = gas_streams0.loc["GTU-KU":"GPK-out", "T"]
Pressure = gas_streams0.loc["GTU-KU", "P"]
gas_streams0.loc["GTU-KU":"GPK-out", "H"] = list(map(lambda x: gas0.p_t(Pressure, x)["h"], Temperatures))




#####################Максимов#####################
PKM_zaryad = True
syngas_streams = pd.read_excel("streams.xlsx", sheet_name="syngas", index_col=0)
##################################################



############################################################
# Задание ГТУ
Gas_turbine = GTU.gtu(GTU_ISO, "GTU-KU")

# Расчет всей ПГУ вместе


def calculate_CCGT(Iterations_KU_TU,Iterations_cotel,Iterations_turbine):

    # Расчет ГТУ
    Gas_turbine_res = Gas_turbine.calc(GTU_input)

    # Запись данных об электричестве
    electric.at["GTU", "N"] = Gas_turbine_res["N"]
    electric.at["GTU", "KPD"] = Gas_turbine_res["eff"]
    electric.at["DK", "N"] = Gas_turbine_res["Ndk"]

    # Запись данных о газе на выходе из ГТУ
    gas_streams = pd.read_excel("streams.xlsx", sheet_name="gas", index_col=0)
    gas_streams.at["GTU-KU", "T"] = Gas_turbine_res["T"]
    gas_streams.at["GTU-KU", "G"] = Gas_turbine_res["G"]
    gas_streams.at["GTU-KU", "P"] = 0.1
    gas_streams.at["GTU-KU", "H"] = gas0.p_t(
        gas_streams.at["GTU-KU", "P"], gas_streams.at["GTU-KU", "T"]
    )["h"]
    Gas_turbine_composition = pd.read_excel(
        "input.xlsx", sheet_name="Gas_composition0", index_col=0
    )
    
    #####################Максимов#####################
    from PKM import PKM_all
    PKM = PKM_all.calc(PKM_zaryad, gas_streams,syngas_streams,water_streams,water_streams0,heaters)
    steamVD_to_turbine = PKM['steamVD_to_turbine']
    ##################################################

    # Состав газов при частичной нагрузке
    fractiongas = list(gas_streams.loc["GTU-PEVD", "N2":"Ar"])

    gas1 = prop.Materials_prop(
        gasmix,
        fractiongas,
        prop.REFPROP_h_s,
        prop.REFPROP_p_t,
        prop.REFPROP_p_h,
        prop.REFPROP_p_s,
        prop.REFPROP_p_q,
        prop.REFPROP_t_q,
        prop.REFPROP_p_rho,
        prop.REFPROP_s_q,
        RP=RP,
    )

    # Инициализаця KU+TU, она здесь потому что нжно менять состав газа на входе в КУ
    
    KU_and_TU = KU_TU.ku_tu(
        gas0,
        gas1,
        water,
        gas_streams0,
        gas_streams,
        water_streams0,
        water_streams,
        heaters,
        electric,
        "PEVD-DROSVD",
        "PPND-DROSND",
        "DROSVD-TURBVD",
        "DROSND-TURBND",
        Calcmethod,
        KPD_SP,
        KPD_KN,
        KPD_to,
        KPD_PN,
        steamVD_fraction_to_turbine,
        steamVD_to_turbine,
    )

    start_time = time.time()

    # Расчет КУ и ТУ
    KU_and_TU.calculate(
        Teplo,
        Calctolerance,
        Iterations_KU_TU,
        Iterations_cotel,
        Iterations_turbine,
    )

#     print(f"fin КУ и ТУ:--- {round((time.time() - start_time), 1)} сек. ---")

Iter_pkm = 100
pkm_pgu_tol = 10**-3
Gst = n.zeros(Iter_pkm)
Qref = n.zeros(Iter_pkm)
Ggpk = n.zeros(Iter_pkm)
for i in range(1,Iter_pkm):
    calculate_CCGT(Maxiterations_KU_TU, Maxiterations_cotel, Maxiterations_turbine)
    Gst[i] = water_streams.at['DROSVD-ST','G']
    Qref[i] = heaters.loc["Ref_all", "Qw"]
    Ggpk[i] = water_streams.at['SMESH-GPK','G']
    
    Err1 = (Gst[i]-Gst[i-1])/(Gst[i])
    Err2 = (Qref[i]-Qref[i-1])/(Qref[i])
    Err3 = (Ggpk[i]-Ggpk[i-1])/(Ggpk[i])
    
    if Err1<pkm_pgu_tol and Err2<pkm_pgu_tol: break
    print(Err1,Err2,Err3)


1.0 1.0 1.0
0.21885879831459218 0.25830665222400756 0.15121852693026788
0.07786318977488454 0.07967119407697751 0.05452713921868802
0.018025139888009953 0.01840986728495455 0.0126770265056086
0.00430151954690675 0.004390799609717007 0.0030301317191741253
0.0009973812802648028 0.0010180402572057773 0.0007344044063667928


In [2]:
gas_streams

,T,P,H,G,N2,O2,CO2,H2O,Ar
AIR,15.0,0.1,NaN,NaN,NaN,NaN,NaN,NaN,NaN
GTU-KU,542.1,0.1,958.869199,503.8,0.7803,0.1237,0.0301,0.0594,0.0065
REF-SMESH,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
GTU-PEVD,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
PEVD-IVD,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
IVD-EVD,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
EVD-PPND,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
PPND-IND,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
IND-GPK,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
GPK-out,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [3]:
water_streams

,T,P,H,G,S,X
AIR,15.000000,0.100000,NaN,NaN,NaN,NaN
PEVD-DROSVD,542.871798,4.170060,3542.396054,83.449310,NaN,NaN
IVD-PEVD,266.588332,5.215256,2792.360002,83.449310,NaN,NaN
EVD-IVD,264.430549,5.215256,1157.057699,83.449310,NaN,NaN
PEN-EVD,146.150001,5.215256,618.611163,83.449310,NaN,NaN
BND-PEN,145.356491,0.419781,612.172045,83.449310,NaN,NaN
PPND-DROSND,188.459485,0.411201,2835.985170,14.654875,NaN,NaN
IND-PPND,145.356491,0.419781,2740.244085,14.654875,NaN,NaN
GPK-IND,145.356491,0.419781,618.117568,98.104185,NaN,NaN
GPK-REC,145.356491,0.419781,618.117568,98.107873,NaN,NaN


In [4]:
heaters

,Qw,Qg,KPD
PEVD,62595.828687,63228.109785,0.99
IVD,136459.983277,137838.366946,0.99
EVD,44933.051420,45386.920626,0.99
PPND,1403.432764,1417.608853,0.99
IND,30600.940330,30910.040737,0.99
GPK,33021.953955,33355.509046,0.99
SP2,NaN,NaN,NaN
SP1,NaN,NaN,NaN
OD,NaN,NaN,NaN
Strans,216379.212872,NaN,NaN


In [5]:
syngas_streams

,T,P,H,G,N2,O2,CO2,Ar,H2O,CH4,H2,CO
REF-COOL,700.0,2.0,3683.821929,73.566634,0.0,0.0,0.063560,0.0,0.536620,0.050040,0.325900,0.023880
COOL-HTS,450.0,2.0,3023.629730,73.566634,0.0,0.0,0.063560,0.0,0.536620,0.050040,0.325900,0.023880
HTS-X,275.0,2.0,2545.703624,73.566634,0.0,0.0,0.086415,0.0,0.513767,0.050042,0.348747,0.001029
